# Teljesítménymérés

Ezen notebook célja olyan mérések elvégzése, amelyek során azt vizsgáljuk, hogy a felbontásnövelő rétegek száma mellett hogyan változik a tanítás hossza.

In [1]:
import time
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

2022-03-20 19:13:04.420035: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-20 19:13:04.420081: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
def make_generator_model(latent_dim, units_per_layer, upsampling_layers, kernel_size):
    noise = keras.layers.Input(shape=[latent_dim])
    hidden = keras.layers.Reshape((1, 1, latent_dim))(noise)

    hidden = keras.layers.Conv2DTranspose(units_per_layer, 4, 1, 'valid')(hidden)
    hidden = keras.layers.BatchNormalization()(hidden)
    hidden = keras.layers.ReLU()(hidden)
    
    for i in range(upsampling_layers):
        hidden = keras.layers.Conv2DTranspose(units_per_layer, kernel_size, 2, 'same')(hidden)
        hidden = keras.layers.BatchNormalization()(hidden)
        hidden = keras.layers.ReLU()(hidden)

    hidden = keras.layers.Conv2D(3, kernel_size, 1, 'same')(hidden)
    out = keras.layers.Activation("tanh")(hidden)
    
    return keras.Model(inputs=noise, outputs=out)

In [3]:
def make_discriminator_model(input_shape, latent_dim, units_per_layer, downsampling_layers, kernel_size):
    image = keras.layers.Input(shape=input_shape)

    hidden = keras.layers.Conv2D(units_per_layer, kernel_size, 1, 'same')(image)
    hidden = keras.layers.BatchNormalization()(hidden)
    hidden = keras.layers.ReLU()(hidden)
    
    for i in range(downsampling_layers):
        hidden = keras.layers.Conv2D(units_per_layer, kernel_size, 2, 'same')(hidden)
        hidden = keras.layers.BatchNormalization()(hidden)
        hidden = keras.layers.ReLU()(hidden)
    
    hidden = keras.layers.Conv2D(latent_dim, 4, 1, 'valid')(hidden)
    hidden = keras.layers.ReLU()(hidden)
    
    hidden = keras.layers.Flatten()(hidden)
    out = keras.layers.Dense(1)(hidden)
    
    return keras.Model(inputs=image, outputs=out)

In [4]:
cross_entropy = keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = keras.optimizers.Adam(1e-4, beta_1=0.5)
discriminator_optimizer = keras.optimizers.Adam(4e-4, beta_1=0.5)

In [5]:
def train_step(generator, discriminator, images):    
    noise = tf.random.normal([batch_size, latent_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)

    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return (gen_loss, disc_loss)

In [6]:
def train(generator, discriminator, dataset, epochs):
    avg_runtime = 0
    for epoch in range(epochs):
        start = time.time()

        for (batch, image_batch) in enumerate(dataset): 
            gen_loss, disc_loss = trainn_step(generator, discriminator, image_batch)

        epoch_runtime = time.time() - start
        avg_runtime += epoch_runtime
    return avg_runtime/epochs

In [7]:
def make_dataset(image_size, batch_size):
    (x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

    x_train = tf.image.resize(x_train, image_size)
    x_train = (x_train - 127.5) / 127.5 #Normalizing

    dataset = tf.data.Dataset.from_tensor_slices(
        x_train
    ).batch(batch_size, drop_remainder=True)
    return dataset

In [8]:
epochs = 5

latent_dim = 100
image_sizes = [(4, 4), (8, 8), (16, 16), (32, 32)]
batch_size = 32
units_per_layer = 128
resizing_layers = [0, 1, 2, 3]
kernel_size = 3

sizes_avg_runtimes = []
for i in range(len(resizing_layers)):
    dataset = make_dataset(image_sizes[i], batch_size)
    
    discriminator = make_discriminator_model(
        input_shape=(image_sizes[i] + (3, )),
        latent_dim=latent_dim,
        units_per_layer=units_per_layer,
        downsampling_layers=resizing_layers[i],
        kernel_size=kernel_size
    )

    generator = make_generator_model(
        latent_dim=latent_dim,
        units_per_layer=units_per_layer,
        upsampling_layers=resizing_layers[i],
        kernel_size=kernel_size
    )

    @tf.function
    def trainn_step(generator, discriminator, images):
        return train_step(generator, discriminator, images)

    avg_runtime = train(generator, discriminator, dataset, epochs)
    print(f"Image size: {image_sizes[i]} with {resizing_layers[i]} resizing layers, Average runtime: {avg_runtime} (s)")
    sizes_avg_runtimes.append(avg_runtime)

2022-03-20 19:13:10.235402: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-20 19:13:10.235442: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-20 19:13:10.235474: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Inspiron-5558): /proc/driver/nvidia/version does not exist
2022-03-20 19:13:10.235875: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-20 19:13:10.778865: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 153600000 exc

Image size: (4, 4) with 0 resizing layers, Average runtime: 23.508806467056274 (s)


2022-03-20 19:15:10.352717: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 153600000 exceeds 10% of free system memory.


Image size: (8, 8) with 1 resizing layers, Average runtime: 90.69318532943726 (s)


2022-03-20 19:22:45.866341: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 153600000 exceeds 10% of free system memory.
2022-03-20 19:22:45.985804: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 153600000 exceeds 10% of free system memory.
2022-03-20 19:22:46.273837: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 153600000 exceeds 10% of free system memory.


Image size: (16, 16) with 2 resizing layers, Average runtime: 335.5662431716919 (s)
Image size: (32, 32) with 3 resizing layers, Average runtime: 1363.9864540576934 (s)


In [9]:
import numpy as np
saved = np.array(sizes_avg_runtimes)

np.save("sizes_avg_runtime_laptop.npy", saved)